# OPEN FOOD NOTEBOOK

In [19]:
import pandas as pd
import numpy as np
import scipy as sp
#import folium
import re
%matplotlib inline

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from pandas.plotting import scatter_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
import seaborn as sns

#from google.cloud import translate
#import pycountry
#import emoji

#translate_client = translate.Client()

from py_translator import Translator

We need to create a data folder where the .csv file will be stored and also a maps folder where .html maps will be stored

In [20]:
data_folder = './data/'
maps_folder = './maps/'

In [66]:
filename = 'en.openfoodfacts.org.products.csv'
countryfile = 'wikipedia-iso-country-codes.csv'

In [37]:
using_col = [
    "product_name",
    "generic_name",
    "quantity",
    "brands",
    "brands_tags",
    "categories",
    "categories_tags",
    "categories_en",
    "manufacturing_places",
    "manufacturing_places_tags",
    "labels",
    "labels_tags",
    "labels_en",
    "purchase_places",
    "countries",
    "countries_tags",
    "countries_en",
    "ingredients_text",
    "allergens",
    "allergens_en",
    "traces",
    "traces_tags",
    "traces_en",
    "nutrition_grade_uk",
    "nutrition_grade_fr",
    "main_category",
    "main_category_en",
    "energy_100g",
    "energy-from-fat_100g",
    "fat_100g",
    "saturated-fat_100g",
    "trans-fat_100g",
    "cholesterol_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
    "sodium_100g",
    "alcohol_100g",
    "calcium_100g",
    "iron_100g",
    "carbon-footprint_100g",
    "nutrition-score-fr_100g",
    "nutrition-score-uk_100g",
    "glycemic-index_100g"
]

In [23]:
food_df = pd.read_csv(data_folder + filename, 
                      sep='\t',
                      header=0,
                      usecols = using_col,
                      quotechar='"', 
                      low_memory=False)

In [24]:
print("The types of the data set are: \n", format(food_df.dtypes))
print ("The total size of the data set is:", format(food_df.shape) )
food_df.head(5)

The types of the data set are: 
 product_name                  object
generic_name                  object
quantity                      object
brands                        object
brands_tags                   object
categories                    object
categories_tags               object
categories_en                 object
origins                       object
origins_tags                  object
manufacturing_places          object
manufacturing_places_tags     object
labels                        object
labels_tags                   object
labels_en                     object
purchase_places               object
countries                     object
countries_tags                object
countries_en                  object
ingredients_text              object
allergens                     object
allergens_en                  object
traces                        object
traces_tags                   object
traces_en                     object
nutrition_grade_uk           float64
nutri

,product_name,generic_name,quantity,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,...,proteins_100g,salt_100g,sodium_100g,alcohol_100g,calcium_100g,iron_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g
0,Vitória crackers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.8,1.40,0.551181,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cacao,NaN,130 g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sauce Sweety chili 0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.2,2.04,0.803150,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mendiants,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Salade de carottes râpées,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.9,0.42,0.165354,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Cleaning Data

## Filling in missing product_name
This section deals with NaN values for `product_name`. If it does not have a `product_name`, the `generic_name` was used. If neither field was filled, then a combination of `brands` and `categories_en`/`main_category` was used.

In [25]:
# remove rows where the columns we are interested in are all null
food_df = food_df.dropna(subset=using_col, how='all')

print("Number of rows w/missing product_name: {}".format(len(food_df) - food_df['product_name'].count()))

Number of rows w/missing product_name: 25881


In [26]:
def get_name(row):
    if pd.isnull(row['product_name']):
        if pd.isnull(row['generic_name']):
            if pd.isnull(row['main_category_en']) & pd.isnull(row['categories_en']) & pd.isnull(row['brands']):
                return
            else:
                category_name = row['main_category_en']
                if pd.isnull(category_name):
                    category_name = row['categories_en']
                return "{} {}".format(row['brands'], category_name)
        else:
            return row['generic_name']
    else:
        return row['product_name']
    
df = food_df.copy()

df['product_name'] = df.apply(
    lambda x: get_name(x),
    axis = 1
)

In [27]:
print("Number of rows w/missing product_name after modifications: {}".format(len(df) - df['product_name'].count()))

Number of rows w/missing product_name after modifications: 22458


In [28]:
# removing the columns that we no longer need
used_col = [
    'generic_name',
    'main_category',
    'main_category_en',
    'brands',
    'brands_tags',
    'categories',
    'categories_tags',
    'categories_en',
]

df = df.drop(used_col, axis=1)

## Filling in Missing Values for Country
This section deals with the missing values for `countries_en`. The `countries_en` column represents the countries where the product is sold. This column is important for our analysis because we want to analyze how viable it is to live in each country based off one's dietary restrictions.

In order to fix these missing values, we decided to first fill the column with values from `purchase_places`, then `manufacturing_places`. We decided to use `purchase_places` because if it was purchased in a certain country, obviously it means the product is sold there as well. As for `manufacturing_places`, we assumed that it is most likely that a product manufactured in a country would be sold there as well.

In [29]:
print("Number of rows w/missing countries_en: {}".format(len(df) - df['countries_en'].count()))

Number of rows w/missing countries_en: 459


In [30]:
translator = Translator()

def translate_country(row):
    if pd.isnull(row['countries_en']):
        alt_country = None
        if pd.notna(row['purchase_places']):
            alt_country = row['purchase_places']
        elif pd.notna(row['manufacturing_places']):
            alt_country = row['manufacturing_places']
            
        # got value from purchase_places or manufacturing_places
        if (not alt_country is None) and pd.notna(alt_country):
            en_alt_country = translator.translate(text=alt_country, dest='en').text
            return en_alt_country
        else:
            return   
    else:
        return row['countries_en']
    
df_1 = df.copy()

df_1['countries_en'] = df_1.apply(
    lambda x: translate_country(x),
    axis = 1
)

In [31]:
print("Number of rows w/missing countries_en: {}".format(len(df_1) - df_1['countries_en'].count()))

Number of rows w/missing countries_en: 322


In [32]:
# drop rows without country
df_1 = df_1.dropna(subset=['countries_en'])


print("Number of rows w/multiple countries: {}".format(
    len(df_1[df_1['countries_en'].str.contains(',')])))

print("Number of total rows: {}".format(len(df_1)))

Number of rows w/multiple countries: 28626
Number of total rows: 693524


In [33]:
# remove columns that are no longer needed
used_col = [
    'purchase_places',
    'manufacturing_places',
    'manufacturing_places_tags',
    'countries_tags',
    'countries',
]

df_1 = df_1.drop(used_col, axis = 1)

,product_name,quantity,origins,origins_tags,labels,labels_tags,labels_en,countries_en,ingredients_text,allergens,...,proteins_100g,salt_100g,sodium_100g,alcohol_100g,calcium_100g,iron_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g
0,Vitória crackers,NaN,NaN,NaN,NaN,NaN,NaN,France,NaN,NaN,...,7.8,1.40,0.551181,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cacao,130 g,NaN,NaN,NaN,NaN,NaN,France,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sauce Sweety chili 0%,NaN,NaN,NaN,NaN,NaN,NaN,France,NaN,NaN,...,0.2,2.04,0.803150,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mendiants,NaN,NaN,NaN,NaN,NaN,NaN,France,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Salade de carottes râpées,NaN,NaN,NaN,NaN,NaN,NaN,France,NaN,NaN,...,0.9,0.42,0.165354,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
# shows that some countries_en are lists
df_1[df_1['countries_en'].notnull() & df_1['countries_en'].str.contains(',')][['countries_en', 'product_name']].head()

,countries_en,product_name
173,"France,United States",Lion Peanut x2
196,"France,United States",Moelleux
205,"France,United States",Pack de 2 Twix
290,"France,United States",Root Beer
426,"France,Germany",Fuzetea schwarzer Tee Zitrone


In [62]:
# converts the list substring into an array
df_2 = df_1.copy()

df_2['countries_en'] = df_2.apply(
    lambda x: x['countries_en'].split(','),
    axis = 1
)

In [65]:
# shows that the countries has been properly split
df_2[df_2.index == 173][['product_name', 'countries_en']]

,product_name,countries_en
173,Lion Peanut x2,"[France, United States]"


In [69]:
# map the countries_en to country codes
country_df = pd.read_csv(data_folder + countryfile, 
                      sep=',',
                      header=0,
                      quotechar='"')

country_df.head()

,English short name lower case,Alpha-2 code,Alpha-3 code,Numeric code,ISO 3166-2
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX
2,Albania,AL,ALB,8,ISO 3166-2:AL
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ
4,American Samoa,AS,ASM,16,ISO 3166-2:AS


In [ ]:
df_3 = df_2.copy()

df_3.countries_en.apply(pd.Series) \
    .merge(df_3, left_index = True, right_index = True) \
    .drop(["countries_en"], axis = 1)

In [71]:
def explode(df, lst_cols, fill_value=''):
    # make sure `lst_cols` is a list
    if lst_cols and not isinstance(lst_cols, list):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)

    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()

    if (lens > 0).all():
        # ALL lists in cells aren't empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, lens)
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .loc[:, df.columns]
    else:
        # at least one list in cells is empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, lens)
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .append(df.loc[lens==0, idx_cols]).fillna(fill_value) \
          .loc[:, df.columns]

df_3 = df_2.copy()
explode(df_3,'countries_en')

df_3[df_3.index >= 173]

,product_name,quantity,origins,origins_tags,labels,labels_tags,labels_en,countries_en,ingredients_text,allergens,...,proteins_100g,salt_100g,sodium_100g,alcohol_100g,calcium_100g,iron_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g
173,Lion Peanut x2,NaN,NaN,NaN,NaN,NaN,NaN,"[France, United States]",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174,Milk Chocolate Peanut Butter Malt Balls,NaN,NaN,NaN,NaN,NaN,NaN,[United States],"Peanut butter coating (evaporated cane juice, ...",NaN,...,7.50,0.09652,0.038000,NaN,0.100,0.00090,NaN,NaN,NaN,NaN
175,Organic Tamari Pumpkin Seed,NaN,NaN,NaN,NaN,NaN,NaN,[United States],"Organic pumpkin seeds, organic wheat free tama...",NaN,...,26.47,0.97028,0.382000,NaN,0.441,0.00794,NaN,NaN,NaN,NaN
176,Yogurt Pretzels,NaN,NaN,NaN,NaN,NaN,NaN,[United States],"Pretzels: enriched flour (refined wheat flour,...",NaN,...,5.00,1.14300,0.450000,NaN,0.100,0.00180,NaN,NaN,NaN,NaN
177,Organic Kamut Flakes,NaN,NaN,NaN,NaN,NaN,NaN,[United States],Organic kamut flakes,NaN,...,10.00,NaN,NaN,NaN,NaN,0.00225,NaN,NaN,NaN,NaN
178,Regular Rolled Oats,NaN,NaN,NaN,NaN,NaN,NaN,[United States],Rolled oats,NaN,...,15.38,NaN,NaN,NaN,0.077,0.00415,NaN,NaN,NaN,NaN
179,Quick Rolled Oats,NaN,NaN,NaN,NaN,NaN,NaN,[United States],Quick rolled oats,NaN,...,15.38,NaN,NaN,NaN,0.077,0.00415,NaN,NaN,NaN,NaN
180,Organic Bulgur,NaN,NaN,NaN,NaN,NaN,NaN,[United States],Organic bulgur.,NaN,...,12.50,0.03048,0.012000,NaN,0.050,0.00270,NaN,NaN,NaN,NaN
181,Organic Toasted Buckwheat Groats,NaN,NaN,NaN,NaN,NaN,NaN,[United States],Organic toasted buckwheat groats,NaN,...,12.20,NaN,NaN,NaN,0.098,0.00263,NaN,NaN,NaN,NaN
182,Organic Rolled Barley Flakes,NaN,NaN,NaN,NaN,NaN,NaN,[United States],Rolled barley,NaN,...,10.71,NaN,NaN,NaN,NaN,0.00257,NaN,NaN,NaN,NaN


In [ ]:
df_2 = df_1.copy()

save_index = list(set(using_col) - set(['countries_en']))

df_2.set_index(save_index).apply(lambda x : x.str.split(',')).stack().apply(pd.Series).stack().unstack(level=2).reset_index(level=[0,1])
df_2.head()

print("Number of total rows after countries_en split {}".format(len(df_2)))

In [39]:
# Processing names
food_df[['categories','categories_tags','categories_en']]
orig_df = food_df[food_df['origins'].notnull() | food_df['origins_tags'].notnull()]
food_df[food_df['generic_name'].isna() & food_df['product_name'].isna()][['product_name','generic_name']]
food_df[food_df['manufacturing_places'].notnull() & food_df['countries'].isna() ][['manufacturing_places','countries']
food_df[food_df['labels'].notnull() | food_df["labels_tags"].notnull() | food_df["labels_en"].notnull()][['labels',"labels_tags","labels_en",]]
food_df[food_df['countries_en'].isna()][['purchase_places','countries_en']]
food_df[food_df['allergens'].notnull()][['allergens','allergens_en']]
                                                                                  